In [ ]:
import os
import re
from llama_cpp import Llama
from faster_whisper import WhisperModel # type: ignore
os.environ["TF_ENABLE_ONEDNN_OPTS"] = '0'

In [ ]:
model_size="large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
"""
Size	Parameters	English-only model	Multilingual model	Required VRAM	Relative speed
tiny	39 M	    tiny.en	            tiny	            ~1 GB	        ~32x
base	74 M	    base.en	            base	            ~1 GB	        ~16x
small	244 M	    small.en	        small	            ~2 GB	        ~6x
medium	769 M	    medium.en	        medium	            ~5 GB	        ~2x
large	1550 M	    N/A	                large	            ~10 GB	        1x
"""

In [ ]:
dir="./"
file="xxx.mp4"
format="lrc" # lrc, srt
file_name=os.path.splitext(file)[0]
context_window=4096
vad_filter=False
language=None
replace_file=True
memory=True
model_path="E:\Download\qwen1_5-14b-chat-q2_k.gguf" # 通义千问1.5-14B-Chat-GGUF
audio=dir+file
llm=None
history_golbal=[{"role": "system", "content": "你是一个翻译专家。现在开始，你将所有的句子都把它翻译为的中文。注意：不要添加原文没有的词语,保证翻译的准确性,不需要去解释句子。"}]

In [ ]:
result = model.transcribe(audio, language=language,vad_filter=vad_filter,vad_parameters=dict(min_silence_duration_ms=500))[0]

In [ ]:
def write_lrc(file,result):
    with open(file,'w',encoding='utf-8') as lrcfile:
        for result in result:
            start=convert_time(result.start)[0]
            end=convert_time(result.end)[0]
            lrcfile.write(f"[{start}]{result.text}\n[{end}]\n")
            print(f"[{start}]{result.text}")

def write_srt(file,result):
    with open(file,'w',encoding='utf-8') as srtfile:
        for i,result in enumerate(result):
            start=convert_time(result.start)[1]
            end=convert_time(result.end)[1]
            srtfile.write(f"{i}\n{start} --> {end}\n{result.text}\n\n")
            print(f"[{start}]{result.text}")

def write(file,result,format):
    if format=="lrc":
        write_lrc(file+".lrc",result)
    elif format=="srt":
        write_srt(file+".srt",result)

def convert_time(seconds):
    # 将秒数转换为毫秒
    milliseconds = int(seconds * 1000)
    hours = milliseconds // 3600000
    minutes = milliseconds // 60000
    seconds = (milliseconds % 60000) // 1000
    millis_1 = milliseconds % 100
    millis_2 = milliseconds % 1000
    # 格式化输出
    return f"{minutes:02d}:{seconds:02d}.{millis_1:02d}",f"{hours:02d}:{minutes:02d}:{seconds:02d},{millis_2:03d}"

write(f"{dir}{file_name}",result,format)

In [ ]:
def load_model():
    global llm
    llm = Llama(
    model_path,
    n_gpu_layers=-1, # Uncomment to use GPU acceleration
    seed=1234, # Uncomment to set a specific seed
    n_ctx=context_window, # Uncomment to increase the context window
)

load_model()

In [ ]:
def translate(text):
    global llm,history_golbal
    history=history_golbal.copy()
    history.append({"role": "user", "content": text})
    flag=True
    while flag:
        try:
            out=llm.create_chat_completion(history)
        except Exception as e:
            if "exceed context window " in str(e):
                del history[1:-1]
                continue
        flag=False
    result=out.get("choices")[0].get("message").get("content")
    history.append({"role": "assistant", "content": result})
    if memory:
        history_golbal=history.copy()
    else:
        del history
    return result

def lrc_tran(lrc_file_path):
    new_lrc_file_path = lrc_file_path.replace('.lrc', ' - 翻译.lrc')
    with open(lrc_file_path, 'r', encoding='utf-8-sig') as lrc_file,\
        open(new_lrc_file_path, 'w', encoding='utf-8-sig') as new_lrc_file:
        lines = lrc_file.readlines()
        for i in range(0, len(lines), 2):
            time_start=re.match(r'\[(\d{2}:\d{2}\.\d{2})\]',lines[i])
            time_end=re.match(r'\[(\d{2}:\d{2}\.\d{2})\]',lines[i+1])
            text = lines[i][time_start.end():].strip()
            time_start = time_start.group(1)
            time_end = time_end.group(1)
            trans=translate(text)
            new_lrc_file.write(f"[{time_start}]{text}\n[{time_start}]{trans}\n")
            print(f"[{time_start}]{trans}\n")

def srt_tran(srt_file_path):
    new_srt_file_path = srt_file_path.replace('.srt', ' - 翻译.srt')
    with open(srt_file_path, 'r', encoding='utf-8-sig') as srt_file,\
        open(new_srt_file_path, 'w', encoding='utf-8-sig') as new_srt_file:
        lines = srt_file.readlines()
        for i in range(0, len(lines), 4):
            trans=translate(lines[i+2].strip())
            new_srt_file.write(f"{lines[i]}{lines[i+1]}{trans}{lines[i+3]}\n")
            print(f"{lines[i+1].strip().split(' --> ')[0]}  {trans}")
            
def tran(file_name,format):
    if format=="lrc":
        lrc_tran(file_name+".lrc")
    elif format=="srt":
        srt_tran(file_name+".srt")

tran(f"{dir}{file_name}",format)

if(replace_file):
    os.remove(f"{dir}{file_name}.{format}")
    os.rename(f"{dir}{file_name} - 翻译.{format}",f"{dir}{file_name}.{format}")